## Inisiasi MySQL

In [2]:
import pymysql
from sqlalchemy import create_engine
import pandas as pd

host = '127.0.0.1'
port = "3306"
user = 'root'
password = 'ugm2018'
database = 'gdelt2'

conn_str = 'mysql+pymysql://{}:{}@{}:{}/{}'.format(user, password, host, port, database);           
engine = create_engine(conn_str)

def run(sql):
    df = pd.read_sql_query(sql,engine)
    return df

def execute(sql):
    return engine.execute(statement=sql)


### Rekap Event Semua Negara

In [4]:
df = run("""
SELECT 
    Year, 
    'Brunei' as `Negara`,
    count(a.GLOBALEVENTID) as `Event` 
FROM events_BX a GROUP BY a.Year
UNION
SELECT 
    Year, 
    'Singapore' as `Negara`,
    count(b.GLOBALEVENTID) as `Event`
FROM events_SN b GROUP BY b.Year
UNION
SELECT 
    Year, 
    'Indonesia' as `Negara`,
    count(c.GLOBALEVENTID) as `Event`
FROM events_ID c WHERE c.Year > 2002 GROUP BY c.Year
UNION
SELECT 
    Year, 
    'Malaysia' as `Negara`,
    count(d.GLOBALEVENTID) as `Event`
FROM events_MY d GROUP BY d.Year
UNION
SELECT 
    Year, 
    'Thailand' as `Negara`,
    count(e.GLOBALEVENTID) as `Event`
FROM events_TH e GROUP BY e.Year
  """)

df.pivot(index='Year', columns='Negara', values='Event')

Negara,Brunei,Indonesia,Malaysia,Singapore,Thailand
Year,,,,,
2003,3429,94254,45484,19375,44730
2004,3492,62982,34215,14573,41532
2005,2879,57228,28480,12669,34378
2006,4153,69311,55874,22487,58777
2007,6228,112987,93401,44432,98821
2008,10453,107503,115591,46420,137789
2009,15272,217017,204828,86564,206031
2010,17142,206536,168911,71025,174722
2011,29959,257256,206746,96265,218747


## Rekap Jumlah Event dan Tone (+/-) Per Tahun

In [8]:
# Negara: Malaysia

run("""
SELECT 
    Year, 
    count(a.GLOBALEVENTID) as `Total Events`, 
    sum(CASE WHEN AvgTone > 0 THEN 1 ELSE 0 END) as `Total Positive Tone`,
    sum(CASE WHEN AvgTone = 0 THEN 1 ELSE 0 END) as `Total Netral Tone`,
    sum(CASE WHEN AvgTone < 0 THEN 1 ELSE 0 END) as `Total Negative Tone`,
    avg(AvgTone) as `Average Tone`,
    max(case when AvgTone >= 0 then AvgTone end) max_positive,
    max(case when AvgTone < 0 then AvgTone end) max_negative
FROM events_MY a GROUP BY Year
  """)

,Year,Total Events,Total Positive Tone,Total Netral Tone,Total Negative Tone,Average Tone,max_positive,max_negative
0,2003,45484,45347.0,137.0,0.0,5.343244,9.99436730095755,None
1,2004,34215,34190.0,25.0,0.0,5.294704,9.97955010224949,None
2,2005,28480,28331.0,133.0,16.0,5.194110,9.97304582210243,-8.29744538231623
3,2006,55874,55691.0,162.0,21.0,5.669558,9.99966341384912,-6.4367816091954
4,2007,93401,93209.0,160.0,32.0,5.624144,9.99775972553868,-7.08812260536398
5,2008,115591,115419.0,149.0,23.0,5.591105,9.99883122954653,-7.63673890608879
6,2009,204828,204335.0,493.0,0.0,5.602364,9.9984374728728,None
7,2010,168911,168433.0,478.0,0.0,5.607923,9.99975511803311,None
8,2011,206746,206061.0,685.0,0.0,5.656953,9.99445061043283,None
9,2012,224607,222890.0,1717.0,0.0,5.220474,9.99966681659918,None


## Rekap Jumlah SOURCE URL per Tahun

In [3]:
# Negara: Malaysia

run("""
SELECT 
    Year, 
    count(a.GLOBALEVENTID) as `Total Events`, 
    count(DISTINCT a.SOURCEURL) as `Unique Source URL`,
    sum(CASE WHEN SOURCEURL = "n/a" OR SOURCEURL IS NULL THEN 1 ELSE 0 END) as `Source URL Kosong`
FROM events_LA a GROUP BY Year
  """)

,Year,Total Events,Unique Source URL,Source URL Kosong
0,0,1,1,0.0
1,2003,10,5,0.0
2,2004,2,2,0.0
3,2005,7,2,0.0
4,2006,18,7,0.0
5,2007,13,5,0.0
6,2008,6,1,0.0
7,2012,209,78,0.0
8,2013,20381,4623,1217.0
9,2014,24866,6104,0.0


## Rekap Kategori Event per Tahun

In [37]:
# Negara: Malaysia

df = run("""
SELECT         
    Year, 
    CONCAT(EventCode,'. ',(SELECT description FROM CAMEO_events_code WHERE code = EventCode)) as Description,
    count(GLOBALEVENTID) as Jumlah
FROM events_MY 
WHERE Year > 2013 
GROUP BY EventCode, Year
HAVING Description IS NOT NULL
  """)

df.pivot(index='Description', columns='Year', values='Jumlah')

Year,2014,2015,2016,2017,2018
Description,,,,,
"010. Make statement, not specified below",47600.0,51869.0,44880.0,40137.0,30209.0
011. Decline comment,900.0,1119.0,1068.0,1017.0,725.0
012. Make pessimistic comment,10095.0,11462.0,9929.0,8544.0,6266.0
013. Make optimistic comment,12520.0,14062.0,11927.0,10875.0,8784.0
014. Consider policy option,7204.0,7108.0,6396.0,5484.0,5538.0
015. Acknowledge or claim responsibility,1587.0,2537.0,2291.0,2161.0,1535.0
016. Deny responsibility,2566.0,2857.0,2762.0,2828.0,2414.0
017. Engage in symbolic act,2669.0,2685.0,2652.0,2603.0,1271.0
018. Make empathetic comment,2041.0,1529.0,1229.0,908.0,433.0
